<a href="https://colab.research.google.com/github/toussaintma/neuralnetworksfromzerotohero/blob/main/walkthrough_makemore_mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [148]:
# data and code at https://github.com/karpathy/makemore
# course at https://www.youtube.com/watch?v=PaCmpygFfXo&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ&index=2&t=11s

In [149]:
import pandas as pd
import torch

In [150]:
df_words = pd.read_csv('names.txt', header=None, names=['name'])

In [151]:
df_words.shape

(32033, 1)

In [152]:
df_words['name']

0            emma
1          olivia
2             ava
3        isabella
4          sophia
           ...   
32028       zylas
32029       zyran
32030       zyrie
32031       zyron
32032       zzyzx
Name: name, Length: 32033, dtype: object

In [153]:
stat = {}
for w in df_words.loc[:, 'name']:
  w = '.' + w + '.'
  for ch1, ch2 in zip(w, w[1:]):
    #print(ch1, ch2, sep='')
    if ch1 + ch2 in stat:
      stat[ch1 + ch2] += 1
    else:
      stat[ch1 + ch2] = 1
len(stat)

627

In [154]:
stat['em']

769

In [155]:
sorted(stat.items(), key = lambda v: -v[1])[:10]

[('n.', 6763),
 ('a.', 6640),
 ('an', 5438),
 ('.a', 4410),
 ('e.', 3983),
 ('ar', 3264),
 ('el', 3248),
 ('ri', 3033),
 ('na', 2977),
 ('.k', 2963)]

In [156]:
def get_index(c):
  result = 0
  if (c != '.'):
    result = ord(c) - ord('a') + 1
  return result

def get_char(i):
  result = '.'
  if (i != 0):
    result = chr(ord('a') + i - 1)
  return result
get_index('e'), get_index('m'), get_index(get_char(4)), get_index('.'), get_char(0)

(5, 13, 4, 0, '.')

In [157]:
stat['.a'], stat['a.']

(4410, 6640)

In [158]:
stats = [[0 for i in range(27)] for j in range(27)]
len(stats)

27

In [159]:
for k, v in stat.items():
  stats[get_index(k[0])][get_index(k[1])] = v
t_stats = torch.tensor(stats, dtype=torch.float32)
t_stats.shape, t_stats.ndim

(torch.Size([27, 27]), 2)

In [160]:
t_stats[5, 13] # is 'em' = 769

tensor(769.)

In [161]:
t_stats.sum(dim=1, keepdim=True).shape

torch.Size([27, 1])

In [162]:
t_stats = t_stats / t_stats.sum(dim=-1).unsqueeze(-1) # a is 556/27245 found 2.0407e-02 OK
t_stats[0]

tensor([0.0000, 0.1377, 0.0408, 0.0481, 0.0528, 0.0478, 0.0130, 0.0209, 0.0273,
        0.0184, 0.0756, 0.0925, 0.0491, 0.0792, 0.0358, 0.0123, 0.0161, 0.0029,
        0.0512, 0.0642, 0.0408, 0.0024, 0.0117, 0.0096, 0.0042, 0.0167, 0.0290])

In [163]:
t_stats.sum(dim=1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

In [164]:
sample = torch.multinomial(t_stats, 20, replacement=True)
result = '.'
next_char = '.'
done = False
for i in range(10):
  next_char = get_char(sample[get_index(next_char)][i])
  if (next_char == '.'):
    done = True
  if (not done):
    result = result + next_char
print(result)

.feyann


In [165]:
t_stats[0]

tensor([0.0000, 0.1377, 0.0408, 0.0481, 0.0528, 0.0478, 0.0130, 0.0209, 0.0273,
        0.0184, 0.0756, 0.0925, 0.0491, 0.0792, 0.0358, 0.0123, 0.0161, 0.0029,
        0.0512, 0.0642, 0.0408, 0.0024, 0.0117, 0.0096, 0.0042, 0.0167, 0.0290])

In [177]:
torch.zeros(1)

tensor([0.])

In [188]:
ll = 0
n = 0
result = []
for w in df_words.loc[:, 'name']:
  w = '.' + w + '.'
  for ch1, ch2 in zip(w, w[1:]):
    newll = torch.log(t_stats[get_index(ch1)][get_index(ch2)])
    n += 1
    ll += newll.item()
    result.append([ch1, ch2, newll, ll])
ll # should be -38.7856 for the first 3 words
ll /= n
nll = - ll
nll

2.4540144946949742